## Random Forest for Spam
### (make sure to have run featurize_spam.py)

In [1]:
import numpy as np
np.random.seed(1)

import scipy.io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from decisiontree import *

import warnings
warnings.filterwarnings("ignore")

import sys
sys.setrecursionlimit(4000)

In [2]:
spam_path = "datasets/spam-dataset/spam_data.mat"
data = scipy.io.loadmat(spam_path)
x = data["training_data"]
x = StandardScaler().fit_transform(x)
pca = PCA(n_components=0.50, whiten=True)
pca_fit = pca.fit(x)
x = pca_fit.transform(x)

y = np.squeeze(data["training_labels"])
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)
x_test = pca_fit.transform(data["test_data"])
print(x.shape,y.shape,x_test.shape)

(3336, 31) (3336,) (5857, 31)


In [3]:
%%time
rf = RandomForest(num_trees=25,header="spam_randomforest")
rf.fit(x_train, y_train)

2019-03-27 02:42:27,990	WARNING worker.py:1406 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-03-27 02:42:27,993	INFO node.py:423 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-03-27_02-42-27_58638/logs.
2019-03-27 02:42:28,104	INFO services.py:363 -- Waiting for redis server at 127.0.0.1:57498 to respond...
2019-03-27 02:42:28,217	INFO services.py:363 -- Waiting for redis server at 127.0.0.1:47475 to respond...
2019-03-27 02:42:28,221	INFO services.py:760 -- Starting Redis shard with 3.44 GB max memory.
2019-03-27 02:42:28,233	INFO services.py:1384 -- Starting the Plasma object store with 5.15 GB memory using /tmp.


CPU times: user 9.2 s, sys: 2.89 s, total: 12.1 s
Wall time: 6min 47s


In [4]:
%%time
train_preds = rf.predict(x_train,"train")
print("training accuracy:",np.mean(train_preds==y_train))
val_preds = rf.predict(x_val,"val")
print("validation accuracy:",np.mean(val_preds==y_val))
rf.predict(x_test,"test")

training accuracy: 0.5472263868065967
validation accuracy: 0.5344311377245509
saved predictions
CPU times: user 2.78 s, sys: 183 ms, total: 2.97 s
Wall time: 3.01 s
